In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc
print(datetime.datetime.now())

2020-02-22 18:14:46.036076


In [2]:
list_dcm_impr_logs=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/DCM_raw_logs_BL/impressions/*")
print(len(list_dcm_impr_logs))
print(set([x.split(".")[1] for x in list_dcm_impr_logs]))
list_dcm_impr_logs.sort()

df_impr_dcm_logs=pd.DataFrame({"file_path":list_dcm_impr_logs})
df_impr_dcm_logs['utc_date']=df_impr_dcm_logs['file_path'].apply(lambda x: x.split("impression_utc_")[1][:8])

588
{'tsv'}


In [3]:
list_lr_impr_returned=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/impressions/*")
print(len(list_lr_impr_returned))
print(set([x.split(".")[1] for x in list_lr_impr_returned]))
list_lr_impr_returned.sort()

df_impr_lr_returned=pd.DataFrame({"file_path":list_lr_impr_returned})
df_impr_lr_returned['utc_date']=df_impr_lr_returned['file_path'].apply(lambda x: x.split("account7252_impression_")[1][:8])


9908
{'tsv'}


In [4]:
list_common_dates=set(df_impr_lr_returned['utc_date'].unique().tolist()).intersection(set(df_impr_dcm_logs['utc_date'].unique().tolist()))
list_common_dates=list(list_common_dates)
list_common_dates.sort()


In [5]:
writer_mapping="/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/UID_IDL_mapping/"
try:
    os.stat(writer_mapping)
except:
    os.mkdir(writer_mapping)

In [6]:
df_uid_idl_mapping=pd.DataFrame()
error_file_list_lr_returned=[]
error_file_list_dcm_logs=[]
i_counter=0
for date in list_common_dates:
    i_counter+=1
    print(i_counter,date,datetime.datetime.now())
    
    df_lr_returned=pd.DataFrame()
    df_dcm_logs=pd.DataFrame()
    
    file_list_lr=df_impr_lr_returned[df_impr_lr_returned['utc_date']==date]['file_path'].tolist()
    file_list_dcm=df_impr_dcm_logs[df_impr_dcm_logs['utc_date']==date]['file_path'].tolist()
    
    for file in file_list_lr:
        try:
            df=pd.read_csv(file,dtype=str,sep="\t")
            df=df[df['Advertiser ID']=="8095847"]
            df_lr_returned=df_lr_returned.append(df)
        except:
            error_file_list_lr_returned.append(file)
            print("error_file_list_lr_returned: ",file)
    
    for file in file_list_dcm:
        try:
            df=pd.read_csv(file,dtype=str,sep="\t")
            df=df[df['Advertiser ID']=="8095847"]
            df_dcm_logs=df_dcm_logs.append(df)
        except:
            error_file_list_dcm_logs.append(file)
            print("error_file_list_dcm_logs: ",file)
            
        
    merge_col_list=df_dcm_logs.columns.tolist()
    for col in ['User ID', 'Partner1 ID',"Partner2 ID"]:
        try:
            merge_col_list.remove(col)
        except:
            pass
    df_lr_returned=df_lr_returned[['Customer_Link']+merge_col_list]
    df_lr_returned=df_lr_returned.drop_duplicates()

    df_lr_returned=df_lr_returned[df_lr_returned['Customer_Link']!="UNMATCHED"]
    
    df_dcm_logs=df_dcm_logs.drop_duplicates()
    
    df_mapping=pd.merge(df_dcm_logs,df_lr_returned,on=merge_col_list,how="inner")
    df_mapping=df_mapping[['Customer_Link','User ID']]
    df_mapping['file_date']=date
    
    df_uid_idl_mapping=df_uid_idl_mapping.append(df_mapping)
    df_uid_idl_mapping=df_uid_idl_mapping.drop_duplicates(['Customer_Link','User ID'])
    

date_min=df_uid_idl_mapping['file_date'].min()
date_max=df_uid_idl_mapping['file_date'].max()

df_uid_idl_mapping.to_csv(writer_mapping+"BL_GoogleID_IDL_mapping_%s_%s_JL_%s.csv" % (str(date_min),str(date_max),str(datetime.datetime.now().date())),index=False)
gc.collect()

1 20180524 2020-02-22 18:14:55.878205
2 20180525 2020-02-22 18:19:15.066847
3 20180526 2020-02-22 18:22:39.290896
4 20180527 2020-02-22 18:25:11.355922
5 20180528 2020-02-22 18:27:34.629029
6 20180529 2020-02-22 18:30:37.083728
7 20180530 2020-02-22 18:34:16.168314
8 20180531 2020-02-22 18:37:29.051399
9 20180601 2020-02-22 18:40:47.445316
10 20180602 2020-02-22 18:44:55.076624
11 20180603 2020-02-22 18:48:18.010241
12 20180604 2020-02-22 18:51:15.114779
13 20180605 2020-02-22 18:57:18.785228
14 20180606 2020-02-22 19:01:14.159802
15 20180607 2020-02-22 19:08:32.819565
16 20180608 2020-02-22 19:12:42.821243
17 20180609 2020-02-22 19:16:02.340274
18 20180610 2020-02-22 19:19:30.323976
19 20180611 2020-02-22 19:21:49.100766
20 20180612 2020-02-22 19:24:48.017122
21 20180613 2020-02-22 19:27:47.961550
22 20180614 2020-02-22 19:31:00.436986
23 20180615 2020-02-22 19:34:04.533881
24 20180616 2020-02-22 19:36:44.701493
25 20180617 2020-02-22 19:39:12.099245
26 20180618 2020-02-22 19:43:17.34

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


166 20181105 2020-02-23 00:29:36.714775
167 20181106 2020-02-23 00:33:21.023765
168 20181107 2020-02-23 00:36:02.317964
169 20181108 2020-02-23 00:38:54.224142
170 20181109 2020-02-23 00:40:10.921079
171 20181110 2020-02-23 00:41:28.451744
172 20181111 2020-02-23 00:42:40.533539
173 20181112 2020-02-23 00:43:45.289137
174 20181113 2020-02-23 00:44:57.027771
175 20181114 2020-02-23 00:46:09.112370
176 20181115 2020-02-23 00:47:20.992722
177 20181116 2020-02-23 00:48:45.162787
178 20181117 2020-02-23 00:50:33.965125
179 20181118 2020-02-23 00:52:13.330876
180 20181119 2020-02-23 00:54:01.475289
181 20181120 2020-02-23 00:56:10.121799
182 20181121 2020-02-23 00:58:03.690466
183 20181122 2020-02-23 00:59:57.410928
184 20181123 2020-02-23 01:01:51.422025
185 20181124 2020-02-23 01:03:57.929528
186 20181125 2020-02-23 01:06:12.245366
187 20181126 2020-02-23 01:08:12.101981
188 20181127 2020-02-23 01:10:10.006953
189 20181128 2020-02-23 01:11:56.287081
190 20181129 2020-02-23 01:13:55.351512


error_file_list_lr_returned:  /home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/impressions/CL_BigLots_dcm_account7252_impression_2019032115_20190321_172335_2107135840_BL_LR_20190322_042049_0000.tsv.gz
error_file_list_lr_returned:  /home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/impressions/CL_BigLots_dcm_account7252_impression_2019032116_20190321_181308_2107163180_BL_LR_20190322_042239_0000.tsv.gz
error_file_list_lr_returned:  /home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/impressions/CL_BigLots_dcm_account7252_impression_2019032117_20190321_215402_2107187731_BL_LR_20190322_041909_0000.tsv.gz
error_file_list_lr_returned:  /home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/impressions/CL_BigLots_dcm_account7252_impression_2019032118_20190321_220116_2107205764_BL_LR_20190322_042051_0000.tsv.gz
error_file_list_lr_retur

432 20190807 2020-02-23 10:08:18.888526
433 20190808 2020-02-23 10:10:02.739307
434 20190809 2020-02-23 10:11:47.270771
435 20190810 2020-02-23 10:13:33.963676
436 20190811 2020-02-23 10:15:16.875581
437 20190812 2020-02-23 10:17:02.280785
438 20190813 2020-02-23 10:18:50.842149
439 20190814 2020-02-23 10:20:47.040041
440 20190815 2020-02-23 10:22:44.123362
441 20190816 2020-02-23 10:24:43.763373
442 20190817 2020-02-23 10:26:38.714029
443 20190818 2020-02-23 10:28:30.287656
444 20190819 2020-02-23 10:30:22.496554
445 20190820 2020-02-23 10:32:13.700989
446 20190821 2020-02-23 10:34:04.927159
447 20190822 2020-02-23 10:35:57.297158
448 20190823 2020-02-23 10:37:54.723998
449 20190824 2020-02-23 10:39:53.710912
450 20190825 2020-02-23 10:41:50.795051
451 20190826 2020-02-23 10:43:47.945829
452 20190827 2020-02-23 10:45:50.608086
453 20190828 2020-02-23 10:48:06.035650
454 20190829 2020-02-23 10:50:17.046970
455 20190830 2020-02-23 10:52:32.238646
456 20190831 2020-02-23 10:54:50.272138


0

In [35]:
print("len(error_file_list_lr_returned)",len(error_file_list_lr_returned))
print("len(error_file_list_dcm_logs)",len(error_file_list_dcm_logs))
df_error_file_list_lr_returned=pd.DataFrame({"file_path":error_file_list_lr_returned})
df_error_file_list_lr_returned.to_csv(writer_mapping+"df_error_file_list_lr_returned.csv",index=False)

len(error_file_list_lr_returned) 25
len(error_file_list_dcm_logs) 0


In [25]:
# looks all right since multiple files exising, and the errro is mostly about the header name
df_impr_lr_returned[df_impr_lr_returned['utc_date']=="20190321"].shape

(46, 2)

In [26]:
writer_mapping

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/UID_IDL_mapping/'

In [27]:
df_uid_idl_mapping=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/UID_IDL_mapping/BL_GoogleID_IDL_mapping_20180524_20191231_JL_2020-02-23.csv",
                               dtype=str)

In [28]:
df_uid_idl_mapping.shape

(30654338, 3)

In [30]:
df_uid_idl_mapping['file_date'].nunique()

575

In [31]:
df_uid_idl_mapping.head(2)

,Customer_Link,User ID,file_date
0,XY1468hPv3C6LUplcc9AOQzxWzY1udaza4Ln32X1741yLNDek,AMsySZYwlmbsf4rsAWEdAeTInPJe,20180524
1,Xi1468eM-c0d-GBhe0kva9Sv6cGyyAH_7WvdJSUyiRQJUq...,AMsySZbGHLhJxKnEGwG4yrTXarSO,20180524


In [32]:
df_uid_idl_mapping['Customer_Link'].nunique()

22902540

In [33]:
df_uid_idl_mapping['User ID'].nunique()

28085115